In [31]:
import cv2
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm
import os

### paths

In [32]:
image_folder = '/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/dataset'
output_file = 'embeddings.pkl'

### Model paths and var initialisations

In [33]:
face_detector = cv2.dnn.readNetFromCaffe(
    '/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/models/deploy.prototxt',
    '/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/models/res10_300x300_ssd_iter_140000.caffemodel'
)

face_recognizer = cv2.dnn.readNetFromTorch('/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/models/openface.nn4.small2.v1.t7')

### Face Detection

In [34]:
def detect_faces(image, detector):
    h, w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), (104.0, 177.0, 123.0), False, False)
    detector.setInput(blob)
    detections = detector.forward()
    faces = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.5:  # Confidence threshold
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype(int)
            startX, startY, endX, endY = max(0, startX), max(0, startY), min(w-1, endX), min(h-1, endY)
            if endX > startX and endY > startY:  # ensuring the box is non-empty
                faces.append((startX, startY, endX, endY))
    return faces


### Feature Extraction

In [35]:
def extract_features(face_image, recognizer):
    if face_image.size == 0:
        return None
    blob = cv2.dnn.blobFromImage(face_image, 1.0 / 255, (96, 96), (0, 0, 0), True, False)
    recognizer.setInput(blob)
    return recognizer.forward()

### Initialising lists and labels

In [36]:
#embeddings: numerical vectors or features extracted from the img
#labels: corresponding names in the form of labels.
embeddings = []
labels = []

### Image Processing

In [38]:
total_images = len(os.listdir("/Volumes/This One Is Totally Infectious/PROJEKTS/FACE ATTENDANCE/Face Recognition using Caffe/dataset"))

with tqdm(total=total_images, desc="Processing images", unit="image") as pbar:
    for image_name in os.listdir(image_folder):
        image_path = os.path.join(image_folder, image_name)
        image = cv2.imread(image_path)
        if image is not None:
            faces = detect_faces(image, face_detector)
            for face in faces:
                x, y, x2, y2 = face
                face_img = image[y:y2, x:x2]
                embedding = extract_features(face_img, face_recognizer)
                if embedding is not None:
                    embeddings.append(embedding.flatten())  # Flatten the embedding
                    labels.append(os.path.splitext(image_name)[0])  # Use the filename (without extension) as the label
        pbar.update(1)

Processing images: 100%|██████████| 7/7 [00:00<00:00, 14.42image/s]


### Saving the embeddings and labels

In [39]:
with open(output_file, 'wb') as f:
    pickle.dump((embeddings, labels), f)